## Selenium
前面几节，我们学习了用requests构造页面请求来爬取静态网页中的信息，以及通过requests.post构造Ajax请求直接获取返回json信息。   
我们在构造请求时会给请求加上浏览器headers,目的就是为了我们的请求模拟浏览器行为，防止被网站的反爬虫策略限制，selenium是一款强大的工具，它可以控制我们的浏览器，这样一来程序的行为就和人类完全一样了。  
通过seleniu可以解决几个问题：
+ 页面内容是由JavaScript动态生成，通过requests请求页面无法获取内容
+ 爬虫被反爬虫策略限制
+ 让程序行为和人一样

In [6]:
from selenium import webdriver
driver=webdriver.Chrome()  # 创建一个 Chrome WebDriver 实例
driver.get("https://www.baidu.com/")  # 运行发现自动打开了Chrome浏览器，并且定向到了百度首页


### 简介
Selenium就是自动化测试工具，如果浏览器安装一个selenium就可以方便的实现web界面。换句话说selenium支持这些浏览器驱动。selenium用来驱动以及Python的对接，Python进行后期的处理。为什么不用直接的浏览器，而用PhantomJS呢？答案是：效率高！

### 模拟提交
下面的代码实现了模拟提交搜索的功能，首先等页面加载完成，然后输入到搜索文本，点击提交。



In [11]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
driver=webdriver.Chrome()
driver.get("http://www.python.org")
assert "Python" in driver.title
elem=driver.find_element_by_name("q") # 通过name这个属性得到请求这个搜索框
elem.send_keys("pycon")  # 输入文本

elem.send_keys(Keys.RETURN)   # 我们可以利用Keys这个类来模拟键盘输入
print(driver.page_source)  # 获取网页渲染后的源代码，输出page_source属性即可


<html class="js no-touch geolocation fontface generatedcontent svg formvalidation placeholder boxsizing retina" lang="en" dir="ltr" style=""><!--<![endif]--><head>
    <meta charset="utf-8">
    <meta http-equiv="X-UA-Compatible" content="IE=edge">

    <link rel="prefetch" href="//ajax.googleapis.com/ajax/libs/jquery/1.8.2/jquery.min.js">

    <meta name="application-name" content="Python.org">
    <meta name="msapplication-tooltip" content="The official home of the Python Programming Language">
    <meta name="apple-mobile-web-app-title" content="Python.org">
    <meta name="apple-mobile-web-app-capable" content="yes">
    <meta name="apple-mobile-web-app-status-bar-style" content="black">

    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <meta name="HandheldFriendly" content="True">
    <meta name="format-detection" content="telephone=no">
    <meta http-equiv="cleartype" content="on">
    <meta http-equiv="imagetoolbar" content="false">

    <script ty

In [14]:
# 测试用例，有了以上 我们当然可以用来些测试样例了
import unittest
class PythonOrgSearch(unittest.TestCase):
    def sefUp(self):
        self.driver=webdriver.Chrome()
    def test_search_in_pythonorg(self):
        driver=self.driver
        driver.get("http://www.python.org")
        self.assertIn("Python",driver.title)
        elem=driver.find_element_by_name("q")
        elem.send_keys("pycon")
        elem.send_keys(Key.RETURN)
    def teatDown(self):
        self.driver.close()

if __name__=="__main__":
    unittest.main()
        

E
ERROR: C:\Users\DjChen\AppData\Roaming\jupyter\runtime\kernel-304c219b-32b2-4c9d-999e-4be756f48b6b (unittest.loader._FailedTest)
----------------------------------------------------------------------
AttributeError: module '__main__' has no attribute 'C:\Users\DjChen\AppData\Roaming\jupyter\runtime\kernel-304c219b-32b2-4c9d-999e-4be756f48b6b'

----------------------------------------------------------------------
Ran 1 test in 0.001s

FAILED (errors=1)


SystemExit: True

c:\program files (x86)\python1\lib\site-packages\IPython\core\interactiveshell.py:3304: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


## 页面交互
我们仅仅抓取了页面没有什么作用，真正要做的是和页面交互，比如点击和输入等。
+ 那么前提就是找到页面中的元素。 driver.find_element_by_id() 
+ 获取元素之后，下一步就是向文本输入内容了，一般首先使用element.clear()来清空输入文本的内容    

### 填充表单
我们已经知道向文本框中输入文字，但是其它表单元素呢，例如下拉选项卡的处理我们可以如下:element=driver.find_element_by_xpath("//select\[@name="name"\]") 。但是这些方法不是十分的有效，webdriver中提供了一个叫Select方法，可以帮助我们完成这些事情

### 元素拖拽
首先要指定被拖动的元素和拖动目标元素，然后利用ActionChains来实现。

### 页面切换
一个浏览器肯定会有很多的窗口，我们肯定要有方法来实现窗口的切换。
+ driver.switch_to_window("WindowsName")     
此外还可以通过windows_handles方法来获取每个窗口的操作对象。
+ for handle in driver.window_handelel: driver.switch_to_window(handle)

### 弹窗处理
当你发生某个事件以后，页面出现了弹窗提示，你怎样来处理这个提示或者获取提示信息呢。通过switch_to_alert()可以获取弹窗对象
+ alert=driver.switch_to_alert()

### 前进后退
driver.forward()
driver.back()


In [18]:
from selenium.webdriver.support.ui import Select
driver=webdriver.Chrome()
driver.get("http://www.baidu.com")
select=Select(driver.find_element_by_name("name"))
select.select_by_index(index)
select.select_by_visible_text("text")
select.select_by_value(value)
# 由上可以见，我们可以根据索引来选择，可以根据值来选择，可以根据文字来选择


# 怎样提交表单
driver.find_element_by_id("submit").click()  # id是属性，是具备一定的功能的

# 元素拖拽
from selenium.webdriver import ActionChains
action_chains=ActionChains(driver)
action_chains.drag_and_drop(element,target).perform()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[name="name"]"}
  (Session info: chrome=80.0.3987.122)


### Cookies处理
为页面添加cookies,用法如下：


In [19]:
driver.get("http://www.baidu.com")
cookie={"name":"foo","value":"bar"}  # 为访问百度增加了一个cookies
driver.add_cookie(cookie)

In [20]:
driver.get("http://baidu.com")
driver.get_cookies()  # 得到访问百度时候的cookies

[{'domain': 'www.baidu.com',
  'httpOnly': False,
  'name': 'foo',
  'path': '/',
  'secure': True,
  'value': 'bar'},
 {'domain': '.baidu.com',
  'httpOnly': False,
  'name': 'H_PS_PSSID',
  'path': '/',
  'secure': False,
  'value': '1453_21096_30823_26350_28704'},
 {'domain': '.baidu.com',
  'expiry': 1614438301.698609,
  'httpOnly': False,
  'name': 'BAIDUID',
  'path': '/',
  'secure': False,
  'value': 'DB33802DE34A7BF561C198B27582D7C0:FG=1'},
 {'domain': '.baidu.com',
  'expiry': 3730385948.698587,
  'httpOnly': False,
  'name': 'BIDUPSID',
  'path': '/',
  'secure': False,
  'value': 'DB33802DE34A7BF58F7F08F0F69CAD1D'},
 {'domain': '.baidu.com',
  'httpOnly': False,
  'name': 'delPer',
  'path': '/',
  'secure': False,
  'value': '0'},
 {'domain': '.baidu.com',
  'expiry': 3730385948.698599,
  'httpOnly': False,
  'name': 'PSTM',
  'path': '/',
  'secure': False,
  'value': '1582902303'},
 {'domain': '.baidu.com',
  'expiry': 1582988702.202891,
  'httpOnly': False,
  'name': 'B